<a href="https://colab.research.google.com/github/kgautam2103/zoom_participant_list_public/blob/main/zoom_participant_list_utility_functions_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def read_worksheet(work_sheet, columns_list):
  work_sheet_data = work_sheet.get_all_values()
  work_sheet_data_df = pd.DataFrame(work_sheet_data, columns=columns_list)
  if work_sheet_data_df.shape[0] > 0:
    work_sheet_data_df = work_sheet_data_df.drop(work_sheet_data_df.index[0])
  
  return work_sheet_data_df


In [ ]:
def double_encode_uuid(meeting_uuid):
  encoded_uuid = urlencode({'id':meeting_uuid})
  double_encoded_uuid = urlencode({'id':encoded_uuid[3:]})
  return double_encoded_uuid[3:]

In [ ]:
def get_participants_list(meeting_uid):
  get_participants_url = report_meetings+meeting_uid+'/participants?page_size='+page_size
  participants_response = requests.get(get_participants_url,headers=auth_headers)
  #print(apac_participants_response.json())
  participants_list = participants_response.json()['participants']
  next_page_token = participants_response.json()['next_page_token']
  while next_page_token != "":
    new_get_participants_url = get_participants_url+'&next_page_token='+next_page_token
    new_participants_response = requests.get(new_get_participants_url,headers=auth_headers)
    next_page_token =  new_participants_response.json()['next_page_token']
    participants_list.extend(new_participants_response.json()['participants'])
  return participants_list

In [ ]:
def get_poll_answer_df(meeting_uid):
  poll_response_url = base_url+'report/meetings/'+meeting_uid+'/polls'
  poll_response = requests.get(poll_response_url,headers = auth_headers)
  response_list = poll_response.json()['questions']
  poll_answer_df = json_normalize(response_list,
                                record_path=['question_details'],
                                meta=['name', 'email',],
                                errors='ignore')
  return poll_answer_df

In [ ]:
def get_poll1_df(poll_answer_df, poll_id):
  poll1_df=pd.DataFrame()
  poll1_df = poll_answer_df[poll_answer_df.polling_id==poll_id]
  if poll1_df.shape[0] > 0:
    poll1_df  = poll1_df.fillna("")
    poll1_df = poll1_df.reset_index()
    poll1_df = poll1_df.drop(columns=['question','polling_id','date_time','email','index'])
    poll1_df = poll1_df[['name','answer']]
  return poll1_df

In [ ]:
def get_poll2_df(poll_answer_df, poll_id):
  poll2_df = pd.DataFrame()
  poll2_df = poll_answer_df[poll_answer_df.polling_id==poll_id]
  if poll2_df.shape[0] > 0:
    poll2_df  = poll2_df.fillna("")
    poll2_df['question'] = poll2_df['question'].apply(lambda x: re.sub('^[\.\d\s]*','',x,flags=re.IGNORECASE))
    poll2_df.drop_duplicates(subset=['name','question'],inplace=True)
    poll2_df = poll2_df.pivot(index="name", columns="question", values="answer")
    poll2_df.reset_index(inplace=True)
    poll2_df[['answer1','answer2','answer3']] = poll2_df[['Will you be joining us next weekend?',
                                          'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you',
                                          'How was your thoughtless state during meditation?']]
    poll2_df=poll2_df.drop(columns=['How was your thoughtless state during meditation?',
                                'Will you be joining us next weekend?',
                                      'Which of the following mid-week followup sessions are convenenient for you? We look forward to see you'])
  return poll2_df

In [ ]:
def get_participants_df(participants_list,event,ref_id_list,ref_name_list,ref_email_list):
  participants_df=pd.DataFrame()
  participants_df = pd.DataFrame(participants_list)
  participants_df = participants_df.drop(columns=['attentiveness_score','failover','customer_key'])
  participants_df = participants_df[participants_df.duration>600]
  participants_df['id'] = participants_df['id'].apply(lambda x: str(random.randint(1,99999999999)) if x=="" else x)
  participants_df = participants_df.drop_duplicates(subset=['id'])
  participants_df['name'] = participants_df['name'].apply(lambda x: re.sub('^sy[\.\s_-]*','',x,flags=re.IGNORECASE))\
  .apply(lambda x: re.sub('[\.\s_-]*sy$','',x,flags=re.IGNORECASE))
  participants_df = participants_df.drop_duplicates(subset=['name'])
  participants_df = participants_df[~participants_df.id.isin(ref_id_list)]
  participants_df = participants_df[~(participants_df.name).str.lower().isin(ref_name_list)]
  participants_df = participants_df[~(participants_df.user_email).str.lower().isin(ref_email_list)]
  participants_df['event'] = event
  participants_df['event_date'] = report_datetime
  participants_df = participants_df[['event','event_date','id','name','user_email', 'join_time', 'leave_time', 'duration']]
  return participants_df

In [ ]:
def get_rolling_3week_df(rolling_3week_df,final_combined_df):
  rolling_3week_df_combined = rolling_3week_df.append(final_combined_df)
  rolling_3week_df_combined['event_date'] = pd.to_datetime(rolling_3week_df_combined['event_date'])
  rolling_3week_df_final = rolling_3week_df_combined[rolling_3week_df_combined.event_date > report_datetime_3week_before]
  rolling_3week_df_final['name']=rolling_3week_df_final['name'].str.lower()
  rolling_3week_df_final.drop_duplicates(subset=['event','event_date','name'],inplace=True)
  return rolling_3week_df_final

In [ ]:
def get_weekend_archive_df(weekend_archive_df,final_combined_df):
  weekend_archive_df_final = weekend_archive_df.append(final_combined_df)
  weekend_archive_df_final['event_date'] = pd.to_datetime(weekend_archive_df_final['event_date'])
  weekend_archive_df_final['name']=weekend_archive_df_final['name'].str.lower()
  return weekend_archive_df_final

In [ ]:
def get_weekend_archive_df_final_filtered_3m(weekend_archive_df_final):
  weekend_archive_df_final_filtered_3m = weekend_archive_df_final[weekend_archive_df_final.event_date > report_datetime_3month_before]
  weekend_archive_df_final_filtered_3m_frequency = weekend_archive_df_final_filtered_3m.groupby(['name']).agg(count=('name',np.count_nonzero))
  weekend_archive_df_final_filtered_3m_frequency.reset_index(inplace=True)
  weekend_archive_df_final_filtered_3m_frequency.sort_values(by = ['count'],ascending=False,inplace=True)
  return weekend_archive_df_final_filtered_3m_frequency

In [ ]:
def get_archive_group_df_max_date_details(weekend_archive_df_final):
  weekend_archive_group_df = weekend_archive_df_final.groupby('name')
  archive_max_date = weekend_archive_group_df.agg(event_date=('event_date',np.max))
  archive_max_date.reset_index(inplace=True)
  archive_max_date_final = archive_max_date[archive_max_date.event_date > report_datetime_3month_before]

  archive_group_df_max_date_details = pd.merge(weekend_archive_df_final,archive_max_date_final,how='inner',on=['name','event_date'])
  archive_group_df_max_date_details.drop_duplicates(subset=['name','event_date'],inplace=True)
  archive_group_df_max_date_details.drop(columns=['join_time','leave_time','duration'],inplace=True)
  return archive_group_df_max_date_details

In [ ]:
def get_updated_sideroom_details(matched_name_list,archive_group_df_max_date_details):
  matched_name_df = pd.DataFrame(matched_name_list,columns=['name','matched_names','score'])
  matched_name_df = matched_name_df[matched_name_df.score>90]
  matched_name_df_final = pd.DataFrame(matched_name_df['matched_names'])
  matched_name_df_final.rename(columns={'matched_names':'name'},inplace=True)
  matched_name_df_final_details = pd.merge(archive_group_df_max_date_details,matched_name_df_final,how='inner',on='name')

  matched_name_df_final_details['event_date'] = matched_name_df_final_details['event_date'].dt.strftime('%Y-%m-%d')
  matched_name_df_final_details['last_event']=matched_name_df_final_details['event_date'].str.slice(0,10)+" "+matched_name_df_final_details['event']
  matched_name_df_final_details.drop(columns=['event','event_date','id'],inplace=True)
  matched_name_df_final_details.sort_values(by = ['name'],inplace=True)
  return matched_name_df_final_details

In [ ]:
def pivot_rolling_3week_df(rolling_3week_df_final):
  group_name_df = rolling_3week_df_final.groupby(['name'])
  count_name = group_name_df.agg(count=('name',np.count_nonzero))
  max_date = group_name_df.agg(latest_date=('event_date',np.max) )
  max_date.reset_index(inplace=True)
  count_name.reset_index(inplace=True)
  final_max_count = pd.merge(count_name,max_date,on='name')
  final_max_count_filtered = final_max_count[final_max_count['count']>1]
  return final_max_count_filtered

In [ ]:
def get_email_name_ref_df(weekend_archive_df_final):
  email_name_ref_df = weekend_archive_df_final[['name','user_email']]
  email_name_ref_df.drop_duplicates()
  nan_value = float("NaN")
  email_name_ref_df.replace("", nan_value, inplace=True)
  email_name_ref_df.dropna(inplace=True)
  return email_name_ref_df

In [ ]:
def get_move2side_final(move2side_name_list,final_max_count_filtered):
  move2side_name_df = pd.DataFrame(move2side_name_list,columns=['name','matched_names','score'])
  move2side_name_df = move2side_name_df[move2side_name_df.score>90]
  move2side_name_df_final = pd.DataFrame(move2side_name_df['matched_names'])
  move2side_name_df_final_list = move2side_name_df_final['matched_names'].tolist()
  final_max_count_filtered = final_max_count_filtered[~final_max_count_filtered.name.isin(move2side_name_df_final_list)]
  final_max_count_filtered.drop(columns=['count'],inplace=True)
  return final_max_count_filtered